In [ ]:
import gradio as gr
import torchxrayvision as xrv
import skimage, torch, torchvision
import requests

In [ ]:
def predict(im,model_choice):
    
    # Prepare the image:
    img = skimage.io.imread(im)
    img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
    img = img.mean(2)[None, ...] # Make single color channel
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    img = transform(img)
    img = torch.from_numpy(img)
    
    model = xrv.models.DenseNet(weights=model_choice)

    outputs = model(img[None,...]) # or outputs = model.features(img[None,...]) 
    labels = dict(zip(model.pathologies,outputs[0].detach().numpy()))
    return labels


model_choices = ["densenet121-res224-all", "densenet121-res224-rsna","densenet121-res224-nih","densenet121-res224-pc","densenet121-res224-chex","densenet121-res224-mimic_nb","densenet121-res224-mimic_ch"]

demo = gr.Interface(fn=predict, 
             inputs=[gr.inputs.Image(type="filepath"),gr.inputs.Dropdown(model_choices)],
             outputs = gr.outputs.Textbox('labels'),
             examples=[["16747_3_1.jpg"]],
             )
             
demo.launch(share=True, show_error=True)